In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-03-06 02:12:41,624 INFO: Initializing external client
2025-03-06 02:12:41,625 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 02:12:42,307 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Fetching data from 2025-02-05 07:12:41.623445+00:00 to 2025-03-06 06:12:41.623445+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.86s) 


In [4]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 08:00:00
1,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,3,2025-03-05 08:00:00
2,6,2,3,2,0,1,3,2,1,1,...,5,3,3,0,0,0,1,8,4,2025-03-05 08:00:00
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,6,2025-03-05 08:00:00
4,4,2,1,4,1,2,1,2,2,2,...,1,0,2,0,2,0,2,1,7,2025-03-05 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,259,2025-03-05 08:00:00
248,4,1,0,3,2,5,0,1,2,0,...,1,0,0,0,1,2,0,0,260,2025-03-05 08:00:00
249,15,14,12,23,23,25,28,23,29,40,...,4,1,0,0,0,2,0,4,261,2025-03-05 08:00:00
250,147,158,114,84,84,65,73,99,89,115,...,1,2,2,4,7,18,79,171,262,2025-03-05 08:00:00


In [5]:
predictions = fetch_next_hour_predictions()

2025-03-06 02:12:51,468 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 02:12:51,469 INFO: Initializing external client
2025-03-06 02:12:51,470 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 02:12:52,008 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.45s) 
Current UTC time: 2025-03-06 07:12:51.468071+00:00
Next hour: 2025-03-06 08:00:00+00:00
Found 0 records


In [6]:
predictions


,pickup_location_id,predicted_demand,pickup_hour


In [7]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [8]:
top10

[]

In [9]:
top10

[]

In [10]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
247    False
248    False
249    False
250    False
251    False
Name: pickup_location_id, Length: 252, dtype: bool

In [11]:
features_cp = features.copy()

In [12]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

ValueError: All arrays must be of the same length

In [ ]:
historical_df